In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil
import collections

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

from scipy.ndimage.filters import gaussian_filter
import cv2

from PIL import ImageFile, ImageOps
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
# GPU Device
gpu_id = 2
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 2: True


# Arguments

In [3]:
source_dir = '/media/data2/dataset/GAN_ImageData/PGGAN_128/'
target_dir = '/media/data2/dataset/GAN_ImageData/StyleGAN2_256/'

In [4]:
pretrained = './log/pggan/128/b0/aug/checkpoint.pth.tar'
resume = ''

In [5]:
# Model
model_name = 'efficientnet-b0' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 1000
start_epoch = 0
train_batch = 250
test_batch = 250
lr = 0.1
momentum = 0.1
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/pggan/128/b0/to_style2/2000shot/self2' # dir
if not os.path.isdir(checkpoint):
    os.makedirs(checkpoint)
num_workers = 8

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

# cutmix
cm_prob = 0.5
cm_beta = 1.0

# augmentation
blur_prob = 0.5
blog_sig = 0.5
jpg_prob = 0.5

best_acc = 0

# sp
sp_alpha = 0.1
sp_beta = 0.1
fc_name = '_fc.'

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['momentum'] = momentum
state['gamma'] = gamma

In [7]:
def data_augment(img):
    img = np.array(img)

    if random.random() < blur_prob:
        sig = np.random.uniform(0.0, 3.0)
        gaussian_blur(img, sig)

    if random.random() < jpg_prob:
        qual = np.random.uniform(30.0, 100.0)
        img = cv2_jpg(img, qual)

    return Image.fromarray(img)


def gaussian_blur(img, sigma):
    gaussian_filter(img[:,:,0], output=img[:,:,0], sigma=sigma)
    gaussian_filter(img[:,:,1], output=img[:,:,1], sigma=sigma)
    gaussian_filter(img[:,:,2], output=img[:,:,2], sigma=sigma)


def cv2_jpg(img, compress_val):
    img_cv2 = img[:,:,::-1]
    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), compress_val]
    result, encimg = cv2.imencode('.jpg', img_cv2, encode_param)
    decimg = cv2.imdecode(encimg, 1)
    return decimg[:,:,::-1]

In [8]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

# Dataset

In [9]:
train_dir = os.path.join(source_dir, 'style2/2000_shot')
val_target_dir = os.path.join(target_dir, 'validation')
val_source_dir = os.path.join(source_dir, 'validation')

train_aug = transforms.Compose([
    transforms.Lambda(lambda img: data_augment(img)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_target_loader = DataLoader(datasets.ImageFolder(val_target_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_source_loader = DataLoader(datasets.ImageFolder(val_source_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [10]:
teacher_model = EfficientNet.from_name(model_name, num_classes=num_classes,
                              override_params={'dropout_rate':0.2, 'drop_connect_rate':0.2})
student_model = EfficientNet.from_name(model_name, num_classes=num_classes,
                              override_params={'dropout_rate':0.2, 'drop_connect_rate':0.2})

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    teacher_model.load_state_dict(torch.load(pretrained)['state_dict'])
    student_model.load_state_dict(torch.load(pretrained)['state_dict'])

=> using pre-trained model './log/pggan/128/b0/aug/checkpoint.pth.tar'


In [11]:
teacher_model.to('cuda')
student_model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in teacher_model.parameters())/1000000.0))

    Total params: 4.01M


In [12]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(student_model.parameters(), lr=lr, momentum=momentum)
scheduler = optim.lr_scheduler.StepLR(optimizer, 200, gamma=0.1, last_epoch=-1)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=2, total_epoch=50, after_scheduler=scheduler_cosine)

# Loss

In [13]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    student_model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Source Loss', 'Train AUROC', 'Valid AUROC', 'Source AUROC'])

In [14]:
for param in teacher_model.parameters():
    param.requires_grad = False
teacher_model.eval()

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_gn0): GroupNorm(8, 32, eps=1e-05, affine=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_gn1): GroupNorm(8, 32, eps=1e-05, affine=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
        (static_padding): Identity()
      )
      (_gn2): GroupNorm(8, 16, ep

In [15]:
teacher_model_weights = {}
for name, param in teacher_model.named_parameters():
    teacher_model_weights[name] = param.detach()

In [16]:
def reg_cls(model):
    l2_cls = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if name.startswith(fc_name):
            l2_cls += 0.5 * torch.norm(param) ** 2
    return l2_cls

def reg_l2sp(model):
    sp_loss = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if not name.startswith(fc_name):
            sp_loss += 0.5 * torch.norm(param - teacher_model_weights[name]) ** 2
    return sp_loss

# Train

In [17]:
def train(train_loader, teacher_model, student_model, criterion, optimizer, epoch, use_cuda):
    teacher_model.eval()
    student_model.train()
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    arc = AverageMeter()
    end = time.time()
    cls_losses = AverageMeter()
    sp_losses = AverageMeter()
    main_losses = AverageMeter()
    alpha = AverageMeter()
    
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
            
        r = np.random.rand(1)
        if cm_beta > 0 and r < cm_prob:
            
            rand_index = torch.randperm(inputs.size()[0]).cuda()
            tt= targets[rand_index]
            boolean = targets==tt
            rand_index = rand_index[boolean]
            lam = np.random.beta(cm_beta, cm_beta)
            bbx1, bby1, bbx2, bby2 = rand_bbox(inputs.size(), lam)
            inputs[boolean, :, bbx1:bbx2, bby1:bby2] = inputs[rand_index, :, bbx1:bbx2, bby1:bby2]

        # compute output
        with torch.no_grad():
            teacher_outputs = teacher_model(inputs)
            teacher_loss = criterion(teacher_outputs, targets)
            sp_alpha = 0
            sigmoid = nn.Sigmoid()
            sp_alpha += sigmoid(-teacher_loss)
        
        outputs = student_model(inputs)
        loss_main = criterion(outputs, targets)
        loss_cls = 0
        loss_sp = 0
        loss_cls = reg_cls(student_model)
        loss_sp = reg_l2sp(student_model)
        loss =  loss_main + 0.1*loss_sp + 0.1*loss_cls

        # measure accuracy and record loss
        losses.update(loss.data.tolist(), inputs.size(0))
        cls_losses.update(loss_cls, inputs.size(0))
        sp_losses.update(loss_sp, inputs.size(0))
        main_losses.update(loss_main.tolist(), inputs.size(0))
        alpha.update(sp_alpha, inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
    print('Train | {batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | Alpha:{alp:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | AUROC:{ac:.4f}'.format(
                     batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, alp=alpha.avg, sp=sp_losses.avg, cls=cls_losses.avg, ac=arc.avg))
    return (losses.avg, arc.avg)

In [18]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    arc = AverageMeter()
    cls_losses = AverageMeter()
    sp_losses = AverageMeter()
    main_losses = AverageMeter()


    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(val_loader):
            # measure data loading time
            data_time.update(time.time() - end)

            if use_cuda:
                inputs, targets = inputs.cuda(), targets.cuda()

            # compute output
            outputs = model(inputs)
            loss_main = criterion(outputs, targets)
            loss_cls = 0
            loss_sp = 0
            loss_cls = reg_cls(model)
            loss_sp = reg_l2sp(model)
            loss = loss_main + 0*loss_sp + 0*loss_cls

            # measure accuracy and record loss
            auroc = roc_auc_score(targets.cpu().detach().numpy(), outputs.cpu().detach().numpy()[:,1])
            losses.update(loss.data.tolist(), inputs.size(0))
            arc.update(auroc, inputs.size(0))
            cls_losses.update(loss_cls, inputs.size(0))
            sp_losses.update(loss_sp, inputs.size(0))
            main_losses.update(loss_main.tolist(), inputs.size(0))


            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

    print('Test | {batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | AUROC:{ac:.4f}'.format(
                     batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, sp=sp_losses.avg, cls=cls_losses.avg, ac=arc.avg))
    return (losses.avg, arc.avg)

In [19]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

In [20]:
for epoch in range(start_epoch, epochs):
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']

    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_auroc = train(train_loader, teacher_model, student_model, criterion, optimizer, epoch, use_cuda)
    test_loss, test_auroc = test(val_target_loader, student_model, criterion, epoch, use_cuda)
    source_loss, source_auroc = test(val_source_loader, student_model, criterion, epoch, use_cuda)


    logger.append([state['lr'], train_loss, test_loss,  source_loss, train_auroc, test_auroc, source_auroc])
    is_best = test_auroc+source_auroc > best_acc
    best_acc = max(test_auroc+source_auroc, best_acc)
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict' : student_model.state_dict(),
        'best_acc': best_acc,
        'optimizer': optimizer.state_dict(),
    }, is_best, checkpoint=checkpoint)
    scheduler_warmup.step()
    scheduler.step()
    
    if (epoch+1)%100 == 0:
        teacher_model.load_state_dict(student_model.state_dict())


Epoch: [1 | 1000] LR: 0.100000
Train | 16/16 | Loss:1.1415 | MainLoss:0.9020 | Alpha:0.0256 | SPLoss:1.0929 | CLSLoss:1.3017 | AUROC:0.0000
Test | 32/16 | Loss:0.6923 | MainLoss:0.6923 | SPLoss:0.9854 | CLSLoss:1.0441 | AUROC:0.5695
Test | 128/16 | Loss:0.6206 | MainLoss:0.6206 | SPLoss:0.9854 | CLSLoss:1.0441 | AUROC:1.0000

Epoch: [2 | 1000] LR: 0.102000
Train | 16/16 | Loss:0.8577 | MainLoss:0.6865 | Alpha:0.0253 | SPLoss:0.8241 | CLSLoss:0.8880 | AUROC:0.0000
Test | 32/16 | Loss:0.6888 | MainLoss:0.6888 | SPLoss:0.6586 | CLSLoss:0.7335 | AUROC:0.5699
Test | 128/16 | Loss:0.5028 | MainLoss:0.5028 | SPLoss:0.6586 | CLSLoss:0.7335 | AUROC:1.0000

Epoch: [3 | 1000] LR: 0.104000
Train | 16/16 | Loss:0.7926 | MainLoss:0.6757 | Alpha:0.0237 | SPLoss:0.5404 | CLSLoss:0.6283 | AUROC:0.0000
Test | 32/16 | Loss:0.6836 | MainLoss:0.6836 | SPLoss:0.4317 | CLSLoss:0.5215 | AUROC:0.5840
Test | 128/16 | Loss:0.2797 | MainLoss:0.2797 | SPLoss:0.4317 | CLSLoss:0.5215 | AUROC:1.0000

Epoch: [4 | 100

Train | 16/16 | Loss:7.5076 | MainLoss:0.3334 | Alpha:0.0240 | SPLoss:71.5865 | CLSLoss:0.1556 | AUROC:0.0000
Test | 32/16 | Loss:0.2784 | MainLoss:0.2784 | SPLoss:53.6209 | CLSLoss:0.1743 | AUROC:0.9561
Test | 128/16 | Loss:0.5341 | MainLoss:0.5341 | SPLoss:53.6210 | CLSLoss:0.1743 | AUROC:0.9783

Epoch: [27 | 1000] LR: 0.152000
Train | 16/16 | Loss:5.0454 | MainLoss:0.2930 | Alpha:0.0240 | SPLoss:47.3544 | CLSLoss:0.1696 | AUROC:0.0000
Test | 32/16 | Loss:0.3733 | MainLoss:0.3733 | SPLoss:36.0319 | CLSLoss:0.1651 | AUROC:0.9593
Test | 128/16 | Loss:0.3027 | MainLoss:0.3027 | SPLoss:36.0320 | CLSLoss:0.1651 | AUROC:0.9595

Epoch: [28 | 1000] LR: 0.154000
Train | 16/16 | Loss:3.1797 | MainLoss:0.2885 | Alpha:0.0250 | SPLoss:28.7387 | CLSLoss:0.1730 | AUROC:0.0000
Test | 32/16 | Loss:0.2673 | MainLoss:0.2673 | SPLoss:22.6017 | CLSLoss:0.1843 | AUROC:0.9652
Test | 128/16 | Loss:0.3841 | MainLoss:0.3841 | SPLoss:22.6017 | CLSLoss:0.1843 | AUROC:0.9748

Epoch: [29 | 1000] LR: 0.156000
Trai

Train | 16/16 | Loss:2.3949 | MainLoss:0.2797 | Alpha:0.0256 | SPLoss:20.9894 | CLSLoss:0.1626 | AUROC:0.0000
Test | 32/16 | Loss:0.2608 | MainLoss:0.2608 | SPLoss:14.7331 | CLSLoss:0.1773 | AUROC:0.9633
Test | 128/16 | Loss:0.8627 | MainLoss:0.8627 | SPLoss:14.7331 | CLSLoss:0.1773 | AUROC:0.9292

Epoch: [52 | 1000] LR: 0.200000
Train | 16/16 | Loss:19.3284 | MainLoss:0.3203 | Alpha:0.0253 | SPLoss:189.9150 | CLSLoss:0.1658 | AUROC:0.0000
Test | 32/16 | Loss:0.6245 | MainLoss:0.6245 | SPLoss:765.6499 | CLSLoss:0.0715 | AUROC:0.8843
Test | 128/16 | Loss:0.7110 | MainLoss:0.7110 | SPLoss:765.6487 | CLSLoss:0.0715 | AUROC:0.6025

Epoch: [53 | 1000] LR: 0.200000
Train | 16/16 | Loss:56.7560 | MainLoss:0.4192 | Alpha:0.0249 | SPLoss:563.2334 | CLSLoss:0.1338 | AUROC:0.0000
Test | 32/16 | Loss:0.2859 | MainLoss:0.2859 | SPLoss:377.3233 | CLSLoss:0.1746 | AUROC:0.9557
Test | 128/16 | Loss:0.5679 | MainLoss:0.5679 | SPLoss:377.3237 | CLSLoss:0.1746 | AUROC:0.9361

Epoch: [54 | 1000] LR: 0.199

Test | 128/16 | Loss:0.5707 | MainLoss:0.5707 | SPLoss:49.0126 | CLSLoss:0.1753 | AUROC:0.9478

Epoch: [76 | 1000] LR: 0.199716
Train | 16/16 | Loss:4.1075 | MainLoss:0.2796 | Alpha:0.0261 | SPLoss:38.1068 | CLSLoss:0.1715 | AUROC:0.0000
Test | 32/16 | Loss:0.2570 | MainLoss:0.2570 | SPLoss:29.3644 | CLSLoss:0.1387 | AUROC:0.9713
Test | 128/16 | Loss:0.8624 | MainLoss:0.8624 | SPLoss:29.3643 | CLSLoss:0.1387 | AUROC:0.8887

Epoch: [77 | 1000] LR: 0.199692
Train | 16/16 | Loss:2.6397 | MainLoss:0.2895 | Alpha:0.0271 | SPLoss:23.3448 | CLSLoss:0.1566 | AUROC:0.0000
Test | 32/16 | Loss:0.2169 | MainLoss:0.2169 | SPLoss:17.7361 | CLSLoss:0.1659 | AUROC:0.9733
Test | 128/16 | Loss:0.6600 | MainLoss:0.6600 | SPLoss:17.7361 | CLSLoss:0.1659 | AUROC:0.9294

Epoch: [78 | 1000] LR: 0.199667
Train | 16/16 | Loss:15.7095 | MainLoss:0.3914 | Alpha:0.0251 | SPLoss:153.0348 | CLSLoss:0.1454 | AUROC:0.0000
Test | 32/16 | Loss:0.3263 | MainLoss:0.3263 | SPLoss:154.4740 | CLSLoss:0.1277 | AUROC:0.9465
T

Test | 32/16 | Loss:0.2570 | MainLoss:0.2570 | SPLoss:189.5985 | CLSLoss:0.1531 | AUROC:0.9645
Test | 128/16 | Loss:0.8735 | MainLoss:0.8735 | SPLoss:189.5985 | CLSLoss:0.1531 | AUROC:0.7464

Epoch: [101 | 1000] LR: 0.198817
Train | 16/16 | Loss:0.4812 | MainLoss:0.2915 | Alpha:0.4349 | SPLoss:1.7409 | CLSLoss:0.1565 | AUROC:0.0000
Test | 32/16 | Loss:0.2100 | MainLoss:0.2100 | SPLoss:2.0563 | CLSLoss:0.1668 | AUROC:0.9768
Test | 128/16 | Loss:0.9472 | MainLoss:0.9472 | SPLoss:2.0563 | CLSLoss:0.1668 | AUROC:0.7438

Epoch: [102 | 1000] LR: 0.198769
Train | 16/16 | Loss:0.4199 | MainLoss:0.2333 | Alpha:0.4352 | SPLoss:1.6967 | CLSLoss:0.1698 | AUROC:0.0000
Test | 32/16 | Loss:0.2031 | MainLoss:0.2031 | SPLoss:1.3900 | CLSLoss:0.1659 | AUROC:0.9803
Test | 128/16 | Loss:1.1294 | MainLoss:1.1294 | SPLoss:1.3900 | CLSLoss:0.1659 | AUROC:0.6997

Epoch: [103 | 1000] LR: 0.198719
Train | 16/16 | Loss:0.3601 | MainLoss:0.2187 | Alpha:0.4339 | SPLoss:1.2495 | CLSLoss:0.1643 | AUROC:0.0000
Test |

...

Test | 32/16 | Loss:0.1584 | MainLoss:0.1584 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.9906
Test | 128/16 | Loss:1.1932 | MainLoss:1.1932 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.8858

Epoch: [901 | 1000] LR: 0.000001
Train | 16/16 | Loss:0.0695 | MainLoss:0.0553 | Alpha:0.4883 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.0000
Test | 32/16 | Loss:0.1584 | MainLoss:0.1584 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.9907
Test | 128/16 | Loss:1.1932 | MainLoss:1.1932 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.8858

Epoch: [902 | 1000] LR: 0.000001
Train | 16/16 | Loss:0.0717 | MainLoss:0.0575 | Alpha:0.4875 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.0000
Test | 32/16 | Loss:0.1584 | MainLoss:0.1584 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.9908
Test | 128/16 | Loss:1.1932 | MainLoss:1.1932 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.8856

Epoch: [903 | 1000] LR: 0.000001
Train | 16/16 | Loss:0.0653 | MainLoss:0.0511 | Alpha:0.4887 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.0000
Test | 32/

Test | 32/16 | Loss:0.1584 | MainLoss:0.1584 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.9907
Test | 128/16 | Loss:1.1932 | MainLoss:1.1932 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.8855

Epoch: [926 | 1000] LR: 0.000001
Train | 16/16 | Loss:0.0551 | MainLoss:0.0410 | Alpha:0.4910 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.0000
Test | 32/16 | Loss:0.1585 | MainLoss:0.1585 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.9908
Test | 128/16 | Loss:1.1931 | MainLoss:1.1931 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.8856

Epoch: [927 | 1000] LR: 0.000001
Train | 16/16 | Loss:0.0714 | MainLoss:0.0572 | Alpha:0.4873 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.0000
Test | 32/16 | Loss:0.1584 | MainLoss:0.1584 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.9908
Test | 128/16 | Loss:1.1932 | MainLoss:1.1932 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.8857

Epoch: [928 | 1000] LR: 0.000001
Train | 16/16 | Loss:0.0741 | MainLoss:0.0599 | Alpha:0.4867 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.0000
Test | 32/

Test | 32/16 | Loss:0.1585 | MainLoss:0.1585 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.9905
Test | 128/16 | Loss:1.1931 | MainLoss:1.1931 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.8856

Epoch: [951 | 1000] LR: 0.000000
Train | 16/16 | Loss:0.0717 | MainLoss:0.0575 | Alpha:0.4876 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.0000
Test | 32/16 | Loss:0.1585 | MainLoss:0.1585 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.9908
Test | 128/16 | Loss:1.1931 | MainLoss:1.1931 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.8856

Epoch: [952 | 1000] LR: 0.000000
Train | 16/16 | Loss:0.0672 | MainLoss:0.0530 | Alpha:0.4880 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.0000
Test | 32/16 | Loss:0.1585 | MainLoss:0.1585 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.9904
Test | 128/16 | Loss:1.1931 | MainLoss:1.1931 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.8856

Epoch: [953 | 1000] LR: 0.000000
Train | 16/16 | Loss:0.0674 | MainLoss:0.0532 | Alpha:0.4882 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.0000
Test | 32/

Test | 32/16 | Loss:0.1585 | MainLoss:0.1585 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.9904
Test | 128/16 | Loss:1.1931 | MainLoss:1.1931 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.8858

Epoch: [976 | 1000] LR: 0.000000
Train | 16/16 | Loss:0.0669 | MainLoss:0.0527 | Alpha:0.4887 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.0000
Test | 32/16 | Loss:0.1585 | MainLoss:0.1585 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.9907
Test | 128/16 | Loss:1.1931 | MainLoss:1.1931 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.8859

Epoch: [977 | 1000] LR: 0.000000
Train | 16/16 | Loss:0.0765 | MainLoss:0.0624 | Alpha:0.4865 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.0000
Test | 32/16 | Loss:0.1585 | MainLoss:0.1585 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.9904
Test | 128/16 | Loss:1.1931 | MainLoss:1.1931 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.8860

Epoch: [978 | 1000] LR: 0.000000
Train | 16/16 | Loss:0.0702 | MainLoss:0.0560 | Alpha:0.4877 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.0000
Test | 32/

Test | 32/16 | Loss:0.1585 | MainLoss:0.1585 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.9906
Test | 128/16 | Loss:1.1931 | MainLoss:1.1931 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.8859
